In [77]:
import numpy as np
import matplotlib.pyplot as plt

import io 
import pandas as pd 
import pyreadr

from load_data import *
from overlap_genes import *

In [79]:
#Load AE3 data
AE3 = pd.read_csv ('../data/merged_data/AE3.csv')
y_AE3 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_AE3.csv')))
print(np.shape(AE3), np.shape(y_AE3))

#Load AE4 data
AE4 = np.array(pd.read_csv ('../data/merged_data/AE4.csv'))
y_AE4 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_AE4.csv')))
print(np.shape(AE4), np.shape(y_AE4))

#Load BIDDY D0 data
D0 = np.array(pd.read_csv ('../data/merged_data/D0.csv'))
y_D0 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_D0.csv')))
print(np.shape(D0), np.shape(y_D0))

#Load BIDDY D0 data
D6 = np.array(pd.read_csv ('../data/merged_data/D6.csv'))
y_D6 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_D6.csv')))
print(np.shape(D6), np.shape(y_D6))

#Load BIDDY D0 data
D15 = np.array(pd.read_csv ('../data/merged_data/D15.csv'))
y_D15 = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_D15.csv')))
print(np.shape(D15), np.shape(y_D15))

#Load Weinreb LK data
LK = np.array(pd.read_csv ('../data/merged_data/LK.csv'))
y_LK  = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_LK.csv')))
print(np.shape(LK), np.shape(y_LK))

#Load Weinreb LSK data
LSK = np.array(pd.read_csv ('../data/merged_data/LSK.csv'))
y_LSK  =np.squeeze(np.array(pd.read_csv('../data/merged_data/y_LSK.csv')))
print(np.shape(LSK), np.shape(y_LSK))

#Load Weinreb LSKmix data
LSKmix = np.array(pd.read_csv ('../data/merged_data/LSKmix.csv'))
y_LSKmix  = np.squeeze(np.array(pd.read_csv('../data/merged_data/y_LSKmix.csv')))
print(np.shape(LSKmix), np.shape(y_LSKmix))

(32285, 334) (333,)
(32285, 1474) (1473,)
(32285, 462) (461,)
(32285, 155) (154,)
(32285, 179) (268,)
(16760, 611) (610,)
(25289, 3384) (781,)


FileNotFoundError: [Errno 2] No such file or directory: '../data/merged_data/LSKmix.csv'

# Fuse all data sets

In [63]:
genes_interest = set(AE3_genes.index) & set(LSKmix.index)
genes_interest = genes_interest & set(LK.index)
genes_interest = list(genes_interest & set(list(LSK.index)))

AE3_df = AE3.loc[genes_interest,:]
AE4_df = AE4.loc[genes_interest,:]
D0_df = D0.loc[genes_interest,:]
D6_df = D6loc[genes_interest,:]
D15_df = D15.loc[genes_interest,:]
LK_df = LK_.loc[genes_interest,:]
LSK_df = LSK.loc[genes_interest,:]
LSKmix_df = LSKmix.loc[genes_interest,:]

(22844, 610)

In [64]:
print(len(list(genes_interest)))
LSKmix_df.shape

22650

In [71]:
#Fuse ormalized data
all_norm = pd.concat([AE3_df, AE4_df,MEF], axis=1)
print(all_norm.shape)

#Fuse families info
y_AE4 += max(y_AE3) #Add the max indice of families in first dataset so that there is no overlap between families indices.
y_all = np.hstack((y_AE3, y_AE4))

y_MEF += max(y_all)
y_all = np.hstack((y_all, y_MEF))

y_LK += max(y_all)
#y_all = np.hstack((y_all, y_LK))

y_LSK += max(y_all)
#y_all = np.hstack((y_all, y_LSK))

y_LSKmix += max(y_all)
#y_all = np.hstack((y_all, y_LSKmix))
print(len(y_all))

(22650, 2599)
2727


In [70]:
#Remove all genes that are not expressed in at least percentage (default 50%) of the all cells
gene_expressed = filter_norm_data(all_norm,0.05)
all_norm = all_norm[gene_expressed]
print(all_norm.shape)

#Store the name of genes of interest
genes_interest = all_norm.index.values #get name of genes
genes_interest = pd.DataFrame(genes_interest)
genes_interest.to_csv('../data/processed_data/allgenes_interest.csv', index=False)

#Convert into array
all_ = np.array(all_norm)

#Create preprocess data 
allcsv = np.c_[all_.T,y_all]
allcsv = pd.DataFrame(MEFcsv)
allcsv.to_csv('../data/processed_data/all.csv', index=False)

(10476, 2599)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 2599 and the array at index 1 has size 2727

In [ ]:
#Add .csv file that remembers which cells from which data set -> give indices
size_AE3 = np.size(np.array(AE3_scran_df))[0]
size_AE4 = np.size(np.array(AE4_scran_df))[0]
size_MEF= np.size(np.array(MEF_scran_df))[0]
size_LK = np.size(np.array(LK_scran_df))[0]
size_LSK = np.size(np.array(LSK_scran_df))[0]
size_LSKmix = np.size(np.array(LSKmix_scran_df))[0]
sizes = [size_AE3, size_AE4, size_MEF, size_LK, size_LSK, size_LSKmix]

sizes = pd.DataFrame(sizes)
sizes.to_csv('../data/processed_data/ind_datasets.csv', index=False)

In [ ]:
#Load preprocess data
all_norm= np.array(pd.read_csv ('../data/processed_data/all.csv'))
y = np.array(all_norm[:,-1],dtype=int)
all_norm = all_norm[:,0:-1]

model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(all_norm,y)

acc = model.score_
print(acc)